<a href="https://colab.research.google.com/github/CShorten/Robust-Vector-Search/blob/main/Robust_Text_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install docarray[full] > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!pip install datasets > /dev/null

In [3]:
from datasets import load_dataset

data_dict = load_dataset("ag_news");

Using custom data configuration default


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
texts = data_dict["train"]["text"]
len(texts)

120000

In [7]:
!pip install -U sentence-transformers > /dev/null

In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

sentence_embeds = []
sentences = ["This is an example sentence"]
embed = model.encode(sentences)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [35]:
from docarray import DocumentArray, Document

sentences = []
text_db = DocumentArray()
for i, text in enumerate(texts):
  if i % 128 == 0 and i > 0:
    sent_embeds = model.encode(sentences)
    for j, embed in enumerate(sent_embeds):
      text_db.append(Document(embedding=embed, text=sentences[j]))
    sentences = []
  sentences.append(text)

In [70]:
demo = "The Los Angeles Lakers win the 2022 NBA basketball championship!"
query = model.encode([demo])

original_top16 = text_db.find(query, limit = 16)

In [71]:
original_top16.contents

['Lakers 115, Kings 99 When the Lakers were dismantled last summer, the Sacramento Kings thought their biggest rivalry was finished. Turns out Kobe Bryant and his new teammates are ready to start it up again.',
 'NBA: LA Lakers 84, Houston 79 The Los Angeles Lakers gave their Coach Rudy Tomjanovich a triumphant return to Houston Saturday night with an 84-79 victory over the Rockets.',
 'Los Angeles Lakers Team Report - December 1 (Sports Network) - The Los Angeles Lakers go for their third straight victory and seventh in nine games this evening, when they battle the listless Chicago Bulls at the United Center.',
 'Lakers Defeat Hawks, 106-90 (AP) AP - Kobe Bryant scored 24 points, Caron Butler added 22, and the Los Angeles Lakers handed Atlanta its third straight lopsided loss, beating the winless Hawks 106-90 Sunday night.',
 'Lakers Defeat Hornets 89-76 (AP) AP - Kobe Bryant had 20 points, six rebounds and seven assists, and the Los Angeles Lakers struggled to an 89-76 victory over t

In [72]:
import random

def random_delete(sent):
  sent_lst = sent.split(" ")
  rnd_idx = random.choice(sent_lst)
  sent_lst.remove(rnd_idx)
  new_sent = ""
  for tok in sent_lst:
    new_sent += tok
    new_sent += " "
  return new_sent

In [73]:
augmented = random_delete(demo)
augmented

'The Los Angeles Lakers win the 2022 basketball championship! '

In [74]:
augmented_query = model.encode([augmented])

new_top16 = text_db.find(augmented_query, limit = 16)

In [88]:
def get_neighbors(org_top, aug_top):
  org_match_ids = set(org_top[:, "id"])
  aug_match_ids = set(aug_top[:, "id"])
  set_union = len(org_match_ids.union(aug_match_ids))
  set_intersection = len(org_match_ids.intersection(aug_match_ids))
  set_metric = set_intersection / (set_union + set_intersection) * 100
  print("The original and augmented query share " + str(set_metric) + " % of their neighbors.")

get_neighbors(original_top16, new_top16)

The original and augmented query share 46.875 % of their neighbors.


<h2> HuggingFace Generation Pipelines </h2>

In [5]:
!pip install transformers > /dev/null

In [78]:
from transformers import pipeline
text2text_generator = pipeline("text2text-generation")

text2text_generator("question: What is 42 ? context: 42 is the answer to life, the universe and everything")

No model was supplied, defaulted to t5-base (https://huggingface.co/t5-base)


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

[{'generated_text': 'the answer to life, the universe and everything'}]

In [84]:
str_template = "question: What is this about? context: "
str_template += demo

text2text_generator(str_template)

[{'generated_text': 'NBA basketball championship'}]

In [99]:
hf_augmented_query = text2text_generator(str_template)[0]["generated_text"] + ": " + demo
encoded_hf_aug_query = model.encode([hf_augmented_query])
hf_aug_top = text_db.find(encoded_hf_aug_query, limit=16)

In [100]:
get_neighbors(original_top16, hf_aug_top)

The original and augmented query share 43.75 % of their neighbors.


<h2> Visualize Neighbors </h2>

In [101]:
original_top16.contents

['Lakers 115, Kings 99 When the Lakers were dismantled last summer, the Sacramento Kings thought their biggest rivalry was finished. Turns out Kobe Bryant and his new teammates are ready to start it up again.',
 'NBA: LA Lakers 84, Houston 79 The Los Angeles Lakers gave their Coach Rudy Tomjanovich a triumphant return to Houston Saturday night with an 84-79 victory over the Rockets.',
 'Los Angeles Lakers Team Report - December 1 (Sports Network) - The Los Angeles Lakers go for their third straight victory and seventh in nine games this evening, when they battle the listless Chicago Bulls at the United Center.',
 'Lakers Defeat Hawks, 106-90 (AP) AP - Kobe Bryant scored 24 points, Caron Butler added 22, and the Los Angeles Lakers handed Atlanta its third straight lopsided loss, beating the winless Hawks 106-90 Sunday night.',
 'Lakers Defeat Hornets 89-76 (AP) AP - Kobe Bryant had 20 points, six rebounds and seven assists, and the Los Angeles Lakers struggled to an 89-76 victory over t

In [102]:
new_top16.contents

['Lakers 115, Kings 99 When the Lakers were dismantled last summer, the Sacramento Kings thought their biggest rivalry was finished. Turns out Kobe Bryant and his new teammates are ready to start it up again.',
 'Los Angeles Lakers Team Report - December 1 (Sports Network) - The Los Angeles Lakers go for their third straight victory and seventh in nine games this evening, when they battle the listless Chicago Bulls at the United Center.',
 'NBA: LA Lakers 84, Houston 79 The Los Angeles Lakers gave their Coach Rudy Tomjanovich a triumphant return to Houston Saturday night with an 84-79 victory over the Rockets.',
 'Lakers Defeat Hawks, 106-90 (AP) AP - Kobe Bryant scored 24 points, Caron Butler added 22, and the Los Angeles Lakers handed Atlanta its third straight lopsided loss, beating the winless Hawks 106-90 Sunday night.',
 'Lakers Defeat Hornets 89-76 (AP) AP - Kobe Bryant had 20 points, six rebounds and seven assists, and the Los Angeles Lakers struggled to an 89-76 victory over t

In [103]:
hf_aug_top.contents

['Lakers 115, Kings 99 When the Lakers were dismantled last summer, the Sacramento Kings thought their biggest rivalry was finished. Turns out Kobe Bryant and his new teammates are ready to start it up again.',
 'NBA: LA Lakers 84, Houston 79 The Los Angeles Lakers gave their Coach Rudy Tomjanovich a triumphant return to Houston Saturday night with an 84-79 victory over the Rockets.',
 'Los Angeles Lakers Team Report - December 1 (Sports Network) - The Los Angeles Lakers go for their third straight victory and seventh in nine games this evening, when they battle the listless Chicago Bulls at the United Center.',
 'Lakers Defeat Hawks, 106-90 (AP) AP - Kobe Bryant scored 24 points, Caron Butler added 22, and the Los Angeles Lakers handed Atlanta its third straight lopsided loss, beating the winless Hawks 106-90 Sunday night.',
 'Pistons send NBA back to drawing board There #39;s going to be a ring ceremony in an NBA city tonight, and by now you probably know it #39;s not going to be in 